In [1]:
!pip install tqdm


In [4]:
import os
trainingImagesDir = '../public/assets/data/training/asl_alphabet_train'
trainingSigns = sorted(os.listdir(trainingImagesDir))
print(trainingSigns)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [5]:
import os 
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [11]:
import cv2
import tensorflow as tf
import mediapipe as mp
import numpy as np
from tqdm import tqdm
import multiprocessing as mproc
import matplotlib.pyplot as plt
import os


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# cv2.ocl.setUseOpenCL(True)


def process_sign_image(hands, sign, fileName, verbose = False):
    tensors = []
    
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    file = os.path.join(trainingImagesDir, './', sign, fileName)
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    out = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    if verbose:
        plt.title("Processsed hands image")
        plt.imshow(out)
    
    results = hands.process(out)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            tensor = np.array([[res.x, res.y, res.z] for res in hand_landmarks.landmark]).flatten()
            
            if verbose:
                print("flattened tensor\n")
                print(tensor)
                print("\n")
            # normalize to -1 -> 1
            tensor = tf.linalg.normalize(tf.constant(tensor), ord=np.inf, axis=None)[0]
            
            if verbose:
                print("normalized tensor\n")
                print(tensor)
                print("\n")
                
            tensors.append(tensor)
    return tensors

def process_sign_images(sign):
    tensors=[]
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:

        IMAGE_FILES = os.listdir(os.path.join(trainingImagesDir, './', sign))
        for idx, fileName in enumerate(IMAGE_FILES):
            tensors.extend(process_sign_image(hands, sign, fileName))

    result = {}
    result['sign'] = sign
    result['tensors'] = tensors
    return result

def error_callback(r):
    print("error_callback " + str(r))
    
def run_parallel():

    # For static images:
    tensor_map={}
    num_signs = len(trainingSigns)
   
    with mproc.Pool(mproc.cpu_count()) as p:
        with tqdm(total=num_signs) as pbar:
            pbar.update(0)
            sign_idx =0 
                
            def process_complete(results):
                tensors = results['tensors']
                sign = results['sign']
                tensor_map[sign] = tensors
                pbar.update(1)
            
            results = []
            for sign in trainingSigns:
                r = p.apply_async(process_sign_images, [sign], callback=process_complete, error_callback=error_callback)
                results.append(r)
            for r in results:
                r.wait()

def run_serial(signs_to_process = None):

    # For static images:
    tensor_map={}
    num_signs = len(trainingSigns) if signs_to_process == None else len(signs_to_process)

    with tqdm(total=num_signs) as pbar:
        pbar.update(0)
        for sign in trainingSigns:
            if signs_to_process == None or sign in signs_to_process:
                print ("Processing " + sign)
                tensors = process_sign_images(sign)['tensors']
                tensor_map[sign] = tensors
                pbar.update(1)

#     for sign in trainingSigns:
#         process_sign(sign)
#         sign_idx = sign_idx + 1
#         pbar.update(1)
    return tensor_map

def run_individual_image(sign, fileName, verbose = False):
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        return process_sign_image(hands, sign, fileName, verbose)


In [14]:
# tensor_map = run_serial(['A'])
tensor_map = run_serial()

  0%|          | 0/29 [00:00<?, ?it/s]

Processing A


  0%|          | 0/29 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [13]:
import pathlib

outdir = './output/v2/'
pathlib.Path(outdir).mkdir(parents=True, exist_ok=True)

for sign in tensor_map:
    arr = np.array(tensor_map[sign])
#     arr = np.array([i[0] for i in tensor_map[sign]])
#     print (np.shape(tensor_map[sign].numpy()))
    np.save(outdir + sign + '.npy', arr)

In [27]:
values = tensor_map['A']
#     arr = np.array(tensor_map[sign])
arr = np.array([i[0] for i in values])
print(arr)

[[ 9.10509866e-01  9.16692881e-01 -1.38889603e-06 ...  9.57289797e-01
   7.15154060e-01 -3.13244163e-02]
 [ 8.39927646e-01  8.08220102e-01 -1.46893172e-06 ...  9.16200810e-01
   6.73624569e-01 -6.20392429e-02]
 [ 2.57284692e-01  1.00000000e+00 -9.67722273e-07 ...  2.86345788e-01
   8.91324910e-01 -4.80609043e-02]
 ...
 [ 8.54770636e-01  7.72019009e-01 -1.42569548e-06 ...  9.32583153e-01
   6.12309841e-01 -6.00563080e-02]
 [ 7.84610517e-01  1.00000000e+00 -1.43675057e-06 ...  7.57687799e-01
   4.93473930e-01 -8.43402234e-02]
 [ 2.71069461e-01  1.00000000e+00 -1.28333887e-06 ...  3.18266794e-01
   8.75387908e-01 -3.50338332e-02]]


In [24]:
tensor = np.array([[res[0], res[1], res[2]] for res in [(1, 2, 3), (4, 5, 6)]]).flatten()

print (tensor)
tensor = tf.linalg.normalize(tf.constant(tensor), ord=np.inf, axis=None)[0]
print (tensor)
print(tensor.numpy())

[1 2 3 4 5 6]
tf.Tensor([0.16666667 0.33333333 0.5        0.66666667 0.83333333 1.        ], shape=(6,), dtype=float64)
[0.16666667 0.33333333 0.5        0.66666667 0.83333333 1.        ]
